In [62]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim  as optim

In [67]:
import numpy as np
X_train_valid = np.load("../Data/X_train_valid.npy")
X_test = np.load("../Data/X_test.npy")
y_train_valid = np.load("../Data/y_train_valid.npy")
y_test = np.load("../Data/y_test.npy")
person_train_valid = np.load("../Data/person_train_valid.npy")
person_test = np.load("../Data/person_test.npy")

### Shape of data

In [68]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))
# [22 EEG channels (0.5-100Hz; notch filtered), 3 EOG channels,
# 250Hz sampling rate, 4 classes, 9 subjects]

# dont use the last 3 of 25 electrodes, which are EOG not EEG

# recommended: test on all 9 EEG datasets, along with 1 classifier
# across all subjects. use dataset augmentation. 

Training/Valid data shape: (2115, 25, 1000)
Test data shape: (443, 25, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [33]:
print(X_train_valid[0])

[[ 5.6640625   2.19726562 -1.3671875  ... -9.91210938 -8.3984375
  -8.25195312]
 [ 9.47265625  4.98046875  2.1484375  ... -7.2265625  -7.421875
  -3.95507812]
 [ 7.51953125  3.85742188  1.31835938 ... -9.03320312 -7.91015625
  -7.6171875 ]
 ...
 [17.08984375 14.6484375  10.7421875  ...  1.46484375 -0.48828125
   1.46484375]
 [15.13671875  7.8125      1.953125   ...  0.48828125 -1.46484375
  -0.9765625 ]
 [ 8.30078125  1.46484375  0.9765625  ...  6.8359375   7.32421875
   6.34765625]]


In [38]:
X = X_train_valid.transpose([1,2,0])
print(X.shape)

(25, 1000, 2115)


In [40]:
print(np.unique(y_train_valid))
Y = []
for y in y_train_valid:
    if y == 769:
        Y.append(0)
    elif y == 770:
        Y.append(1)
    elif y == 771:
        Y.append(2)
    elif y == 772:
        Y.append(3)

[769 770 771 772]


In [47]:
print(X_train_valid[0][0]) # (1000,)

[  5.6640625    2.19726562  -1.3671875   -2.734375     0.73242188
   1.90429688  -0.68359375   1.80664062   3.27148438  -0.63476562
   2.00195312  -0.43945312   1.171875     1.02539062   3.46679688
  -1.22070312   7.2265625    0.04882812  -4.15039062  -4.44335938
  -9.86328125 -15.28320312  -5.6640625   -3.515625   -10.30273438
  -2.78320312  -8.69140625  -8.3984375   -8.74023438  -1.70898438
  -1.41601562  -8.49609375 -13.23242188 -14.79492188  -8.05664062
  -4.58984375   3.36914062  13.42773438   8.15429688   8.64257812
   3.125        0.34179688  14.2578125   10.05859375  16.50390625
  19.43359375   8.984375     4.15039062   2.39257812  -5.12695312
   4.8828125    0.1953125   -3.125        0.34179688  -4.93164062
   2.88085938   0.68359375   4.73632812  10.44921875   2.1484375
   1.22070312   5.12695312   9.66796875   3.3203125    2.44140625
   0.92773438  -5.81054688  -2.29492188  -4.34570312   2.68554688
   7.32421875   0.34179688   5.81054688   5.6640625    0.68359375
  -0.683593

In [53]:
print((X[0][0:1]).shape)

(1, 2115)


In [54]:
N,C,H,W = 2115,1,25,1000
x = Variable(torch.tensor(X_train_valid.reshape((N,C,H,W))))
y = Variable(torch.tensor(Y),requires_grad = False)
dtype = torch.FloatTensor
x.type(dtype)
y.type(dtype)

tensor([2., 3., 0.,  ..., 3., 3., 2.])

In [55]:
class Flatten(nn.Module):
    def forward(self, x):
        a= x.view(x.size(0), -1)
        print(a.shape)
        return a

In [56]:
model = nn.Sequential(
        nn.Conv2d(1,25,kernel_size=(5,501),stride = 10),
        Flatten(),
        nn.Linear(3750,10))

In [57]:
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)

In [59]:
for t in range(3):
    y_pred = model(x.float())
    loss = loss_fn(y_pred, y.type(torch.LongTensor))
    print('loss:', loss)
    model.zero_grad()
    loss.backward()
loss = loss_fn(y_pred,y.type(torch.LongTensor))

torch.Size([2115, 3750])
loss: tensor(5.5453, grad_fn=<NllLossBackward>)
torch.Size([2115, 3750])
loss: tensor(5.5453, grad_fn=<NllLossBackward>)
torch.Size([2115, 3750])
loss: tensor(5.5453, grad_fn=<NllLossBackward>)


In [61]:
print(loss.data)

tensor(5.5453)
